In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import gc

In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
batch_size = 128
image_size = 64
nz = 100  # Size of latent vector
ngf = 64  # Generator feature maps
ndf = 64  # Discriminator feature maps
lr = 0.0002
beta1 = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_path = r"C:\Users\krish\Downloads\archive (10)\images"

In [4]:
class AnimeFaceDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = [os.path.join(root, f) for f in os.listdir(root) if f.endswith(('png', 'jpg', 'jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [5]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = AnimeFaceDataset(root=dataset_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
# Generator
generator = nn.Sequential(
    nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
    nn.BatchNorm2d(ngf * 8),
    nn.ReLU(True),
    
    nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf * 4),
    nn.ReLU(True),
    
    nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf * 2),
    nn.ReLU(True),
    
    nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ngf),
    nn.ReLU(True),
    
    nn.ConvTranspose2d(ngf, 3, 4, 2, 1, bias=False),
    nn.Tanh()
).to(device)

In [7]:
# Discriminator
discriminator = nn.Sequential(
    nn.Conv2d(3, ndf, 4, 2, 1, bias=False),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf * 2),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf * 4),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
    nn.BatchNorm2d(ndf * 8),
    nn.LeakyReLU(0.2, inplace=True),
    
    nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
    nn.Sigmoid()
).to(device)

In [8]:
criterion = nn.BCELoss()
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))

In [9]:
# Training Loop
epochs = 50
for epoch in range(epochs):
    for i, real_images in enumerate(dataloader):
        real_images = real_images.to(device)
        batch_size = real_images.size(0)
        
        # Train Discriminator
        optimizerD.zero_grad()
        labels_real = torch.ones(batch_size, device=device)
        labels_fake = torch.zeros(batch_size, device=device)
        
        output_real = discriminator(real_images).view(-1)
        loss_real = criterion(output_real, labels_real)
        
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake_images = generator(noise)
        output_fake = discriminator(fake_images.detach()).view(-1)
        loss_fake = criterion(output_fake, labels_fake)
        
        lossD = loss_real + loss_fake
        lossD.backward()
        optimizerD.step()
        
        # Train Generator
        optimizerG.zero_grad()
        output_fake = discriminator(fake_images).view(-1)
        lossG = criterion(output_fake, labels_real)
        lossG.backward()
        optimizerG.step()
        
        if i % 100 == 0:
            print(f'Epoch [{epoch}/{epochs}], Step [{i}/{len(dataloader)}], LossD: {lossD.item():.4f}, LossG: {lossG.item():.4f}')
    
    # Save generated images
    output_dir = "output"
    os.makedirs(output_dir, exist_ok=True)
    vutils.save_image(fake_images, f"{output_dir}/fake_samples_epoch_{epoch}.png", normalize=True)

Epoch [0/50], Step [0/497], LossD: 1.4251, LossG: 2.2791
Epoch [0/50], Step [100/497], LossD: 0.4348, LossG: 3.5041
Epoch [0/50], Step [200/497], LossD: 0.8391, LossG: 5.6006
Epoch [0/50], Step [300/497], LossD: 0.6620, LossG: 6.5078
Epoch [0/50], Step [400/497], LossD: 0.3409, LossG: 4.5688
Epoch [1/50], Step [0/497], LossD: 0.2983, LossG: 5.4858
Epoch [1/50], Step [100/497], LossD: 0.2084, LossG: 5.4767
Epoch [1/50], Step [200/497], LossD: 0.5317, LossG: 3.2868
Epoch [1/50], Step [300/497], LossD: 0.9731, LossG: 7.6769
Epoch [1/50], Step [400/497], LossD: 0.3321, LossG: 6.2790
Epoch [2/50], Step [0/497], LossD: 0.4118, LossG: 4.3672
Epoch [2/50], Step [100/497], LossD: 0.3976, LossG: 6.9068
Epoch [2/50], Step [200/497], LossD: 0.4400, LossG: 5.3785
Epoch [2/50], Step [300/497], LossD: 0.7516, LossG: 7.7937
Epoch [2/50], Step [400/497], LossD: 0.2633, LossG: 4.2789
Epoch [3/50], Step [0/497], LossD: 0.0514, LossG: 5.0137
Epoch [3/50], Step [100/497], LossD: 0.2407, LossG: 4.6424
Epoch

Epoch [27/50], Step [400/497], LossD: 0.1508, LossG: 4.3178
Epoch [28/50], Step [0/497], LossD: 0.0408, LossG: 5.4147
Epoch [28/50], Step [100/497], LossD: 0.1087, LossG: 4.3643
Epoch [28/50], Step [200/497], LossD: 0.7451, LossG: 1.8126
Epoch [28/50], Step [300/497], LossD: 0.1204, LossG: 4.1689
Epoch [28/50], Step [400/497], LossD: 0.0867, LossG: 4.8186
Epoch [29/50], Step [0/497], LossD: 0.1034, LossG: 5.0871
Epoch [29/50], Step [100/497], LossD: 0.0806, LossG: 5.8134
Epoch [29/50], Step [200/497], LossD: 0.5582, LossG: 12.8214
Epoch [29/50], Step [300/497], LossD: 0.0821, LossG: 4.5491
Epoch [29/50], Step [400/497], LossD: 0.3104, LossG: 5.5814
Epoch [30/50], Step [0/497], LossD: 0.0853, LossG: 4.5361
Epoch [30/50], Step [100/497], LossD: 0.1033, LossG: 3.4002
Epoch [30/50], Step [200/497], LossD: 0.1205, LossG: 3.6018
Epoch [30/50], Step [300/497], LossD: 0.0954, LossG: 5.0119
Epoch [30/50], Step [400/497], LossD: 0.0653, LossG: 4.7867
Epoch [31/50], Step [0/497], LossD: 0.2196, L

In [10]:
# Save models
torch.save(generator.state_dict(), 'AnimeFace_generator.pth')
torch.save(discriminator.state_dict(), 'AnimeFace_discriminator.pth')

In [11]:
for _ in range(5):
    noise = torch.randn(16, nz, 1, 1, device=device)
    samples = generator(noise)
    vutils.save_image(samples, f"{output_dir}/sample_{_}.png", normalize=True)
    print(f'Sample {_} generated.')

Sample 0 generated.
Sample 1 generated.
Sample 2 generated.
Sample 3 generated.
Sample 4 generated.
